# Data Augmentation

In [19]:
from PIL import Image, ImageEnhance, ImageOps
import shutil
import os
import cv2
import numpy as np

Debemos de definir los folders de input y output para el proceso de data augmentation. Debemos de mencionar que aqui hay un folder intermedio pues aqui guardamos las imagenes resultantes de dos tipos de augmentation para luego al conjunto completo aplicarle más filtros de augmentation.

In [20]:
input_folder = 'datasets/cropped_and_labeled_final/images'
labels_folder = 'datasets/cropped_and_labeled_final/labels'
intermediate_images_folder = 'datasets/intermediate_augmented/images'
intermediate_labels_folder = 'datasets/intermediate_augmented/labels'

In [21]:
final_images_folderV1 = 'datasets/final_dataset_v1/images'
final_labels_folderV1 = 'datasets/final_dataset_v1/labels'

Nos aseguramos que existan estas direcciones o de lo contrario las creamos para poder almacenar el output del proceso de data augmentation.

In [22]:
if not os.path.exists(intermediate_images_folder):
    os.makedirs(intermediate_images_folder)

if not os.path.exists(intermediate_labels_folder):
    os.makedirs(intermediate_labels_folder)

if not os.path.exists(final_images_folderV1):
    os.makedirs(final_images_folderV1)

if not os.path.exists(final_labels_folderV1):
    os.makedirs(final_labels_folderV1)

In [23]:
def count_images_in_folder(folder_path):
    return len([file for file in os.listdir(folder_path) if file.endswith('.jpg')])

Tenemos que definir las transformaciones que vamos a estar realizando. Para ello primero las imagenes pasaran por un proceso de flip y rotation donde estaremos rotando y girando en multiples ángulos las imagenes.

In [24]:
angles = [-45, 0, 45]  # Rotaciones en grados

Luego de esto tenemos un set de imagenes que han sido rotadas y a estar les aplicaremos los filtros de color, brillo, ruido y demás para enriquecer nuestro set.

In [25]:
def first_phase_augmentation(input_folder, labels_folder, intermediate_images_folder, intermediate_labels_folder):
    initial_image_count = count_images_in_folder(input_folder)
    print(f"Cantidad inicial de imágenes: {initial_image_count}")

    images = [os.path.join(input_folder, file) for file in os.listdir(input_folder) if file.endswith('.jpg')]

    for img_path in images:
        img = Image.open(img_path)
        img_name = os.path.basename(img_path).split('.')[0]
        label_path = os.path.join(labels_folder, f"{img_name}.txt")

        # Copiar la etiqueta original a la carpeta de labels antes de la augmentación
        shutil.copy(label_path, os.path.join(intermediate_labels_folder, f"{img_name}.txt"))

        for angle in angles:
            # Aplicar rotación sin flip
            rotated_img = img.rotate(angle)

            # Guardar la imagen augmentada
            output_filename = f"{img_name}_rot_{angle}.jpg"
            output_image_path = os.path.join(intermediate_images_folder, output_filename)
            rotated_img.save(output_image_path)

            # Guardar el archivo de label con el mismo nombre que la imagen
            output_label_filename = f"{img_name}_rot_{angle}.txt"
            output_label_path = os.path.join(intermediate_labels_folder, output_label_filename)
            shutil.copy(label_path, output_label_path)  # Copiar el mismo label de la imagen original
            # print(f"Imagen y label guardados: {output_filename}, {output_label_filename}")

    final_image_count_fase1 = count_images_in_folder(intermediate_images_folder)
    print(f"Cantidad inicial de imágenes: {final_image_count_fase1}")


In [26]:
def second_phase_augmentation(intermediate_images_folder, intermediate_labels_folder, final_images_folder, final_labels_folder):
    images = [os.path.join(intermediate_images_folder, file) for file in os.listdir(intermediate_images_folder) if file.endswith('.jpg')]

    for img_path in images:
        img = Image.open(img_path)
        img_name = os.path.basename(img_path).split('.')[0]
        label_path = os.path.join(intermediate_labels_folder, f"{img_name}.txt")

        # Copiar la imagen original y su label antes de hacer augmentación
        shutil.copy(img_path, os.path.join(final_images_folder, f"{img_name}.jpg"))
        shutil.copy(label_path, os.path.join(final_labels_folder, f"{img_name}.txt"))

        # Ajustes de brillo
        high_brightness = ImageEnhance.Brightness(img).enhance(1.5)
        low_brightness = ImageEnhance.Brightness(img).enhance(0.5)

        # Filtros de color
        light_gray_filter = ImageEnhance.Brightness(ImageOps.grayscale(img)).enhance(1.2)

        # Ruido para simular gotas de lluvia
        noisy_img = np.array(img)
        noise = np.random.normal(0, 25, noisy_img.shape).astype(np.uint8)
        noisy_img = cv2.addWeighted(noisy_img, 0.8, noise, 0.2, 0)
        noisy_img = Image.fromarray(noisy_img)

        # Aumento y disminución de tamaño (resizing)
        resized_large = img.resize((int(img.width * 1.2), int(img.height * 1.2)))  # Aumentar tamaño 20%
        resized_small = img.resize((int(img.width * 0.8), int(img.height * 0.8)))  # Disminuir tamaño 20%

        # Guardar todas las versiones augmentadas
        transformations = {
            "high_brightness": high_brightness,
            "low_brightness": low_brightness,
            "light_gray_filter": light_gray_filter,
            "rain_noise": noisy_img,
            "resized_large": resized_large,
            "resized_small": resized_small,
        }

        for name, transformed_img in transformations.items():
            output_image_filename = f"{img_name}_{name}.jpg"
            output_image_path = os.path.join(final_images_folder, output_image_filename)
            transformed_img.save(output_image_path)

            # Copiar el mismo label de la imagen original
            output_label_filename = f"{img_name}_{name}.txt"
            output_label_path = os.path.join(final_labels_folder, output_label_filename)
            shutil.copy(label_path, output_label_path)  # Copiar el label original
            # print(f"Imagen y label guardados: {output_image_filename}, {output_label_filename}")

    # Contar las imágenes resultantes
    final_image_count = count_images_in_folder(final_images_folder)
    print(f"Cantidad final de imágenes: {final_image_count}")

In [27]:
first_phase_augmentation(input_folder, labels_folder, intermediate_images_folder, intermediate_labels_folder)

Cantidad inicial de imágenes: 4569
Cantidad inicial de imágenes: 13707


In [28]:
second_phase_augmentation(intermediate_images_folder, intermediate_labels_folder, final_images_folderV1, final_labels_folderV1)

Cantidad final de imágenes: 95949


Vamos a ver cuales son las unicas labels que tenemos en nuestro dataset

In [29]:
# Directorio de labels
labels_folder = 'datasets/cropped_and_labeled_final/labels'

# Crear un conjunto para almacenar las etiquetas únicas
unique_labels = set()

# Leer todos los archivos .txt en el directorio de labels
for label_filename in os.listdir(labels_folder):
    label_path = os.path.join(labels_folder, label_filename)

    # Verificar si es un archivo .txt
    if label_filename.endswith('.txt'):
        with open(label_path, 'r') as label_file:
            # Leer el contenido del archivo (cada archivo puede contener una o más etiquetas)
            label_data = label_file.read().strip()
            # Agregar la etiqueta al conjunto (se evita la duplicación automáticamente)
            unique_labels.add(label_data)

# Convertir el conjunto en una lista para obtener un arreglo
unique_labels_list = list(unique_labels)

# Imprimir las etiquetas únicas
print("Etiquetas únicas presentes en los archivos .txt:")
print(unique_labels_list)

Etiquetas únicas presentes en los archivos .txt:
['traffic_light_red', 'traffic_light_green', 'stop sign', 'traffic_light_yellow', 'pedestrian_traffic_light_red', 'stop', 'pedestrian_traffic_light_green']


Ahora contamos con un set de datos de 95,949 imagenes.

In [30]:
# Función para leer archivos de etiquetas
def read_label_file(label_file):
    labels = []
    with open(label_file, 'r') as f:
        for line in f.readlines():
            class_name = line.split()[0]  # Asumimos que la clase está en la primera columna
            labels.append(class_name.strip())  # Normalizamos eliminando espacios
    return labels

# Función para contar ocurrencias de cada clase
def count_class_occurrences(labels_folder):
    class_counts = {}

    # Recorrer todos los archivos de etiquetas
    for label_file in os.listdir(labels_folder):
        if label_file.endswith('.txt'):
            label_path = os.path.join(labels_folder, label_file)
            labels = read_label_file(label_path)

            # Contar las clases en cada archivo de etiquetas
            for class_name in labels:
                if class_name in class_counts:
                    class_counts[class_name] += 1
                else:
                    class_counts[class_name] = 1

    # Mostrar el conteo de cada clase
    for class_name, count in class_counts.items():
        print(f"Clase '{class_name}': {count} ocurrencias")

    return class_counts

# Uso del código
labels_folder = 'datasets/cropped_and_labeled_final/labels'
class_counts = count_class_occurrences(labels_folder)

Clase 'pedestrian_traffic_light_green': 1272 ocurrencias
Clase 'pedestrian_traffic_light_red': 1256 ocurrencias
Clase 'traffic_light_red': 216 ocurrencias
Clase 'stop': 1018 ocurrencias
Clase 'traffic_light_green': 725 ocurrencias
Clase 'traffic_light_yellow': 82 ocurrencias


## Posterior a analisis y visualización de dataset

Haremos un cambio en el proceso de data augmentation para reducir la cantidad de filtros y variaciones que le estamos aplicando para reducir los filtros que no representan o no añaden tanto valor al set de datos.

In [31]:
input_folder = 'datasets/cropped_and_labeled_final/images'
labels_folder = 'datasets/cropped_and_labeled_final/labels'
final_images_folder = 'datasets/final_output_augmentation/images'
final_labels_folder = 'datasets/final_output_augmentation/labels'

In [32]:
if not os.path.exists(final_images_folder):
    os.makedirs(final_images_folder)

if not os.path.exists(final_labels_folder):
    os.makedirs(final_labels_folder)

In [33]:
def apply_augmentations(img, img_name, label_path, final_images_folder, final_labels_folder):
    # Ajustes de brillo
    high_brightness = ImageEnhance.Brightness(img).enhance(1.5)
    low_brightness = ImageEnhance.Brightness(img).enhance(0.5)

    # Ruido para simular gotas de lluvia
    noisy_img = np.array(img)
    noise = np.random.normal(0, 25, noisy_img.shape).astype(np.uint8)
    noisy_img = cv2.addWeighted(noisy_img, 0.8, noise, 0.2, 0)
    noisy_img = Image.fromarray(noisy_img)

    # Aumento y disminución de tamaño (resizing)
    resized_large = img.resize((int(img.width * 1.2), int(img.height * 1.2)))  # Aumentar tamaño 20%
    resized_small = img.resize((int(img.width * 0.8), int(img.height * 0.8)))  # Disminuir tamaño 20%

    # Guardar todas las versiones augmentadas
    transformations = {
        "high_brightness": high_brightness,
        "low_brightness": low_brightness,
        "rain_noise": noisy_img,
        "resized_large": resized_large,
        "resized_small": resized_small,
    }

    for name, transformed_img in transformations.items():
        output_image_filename = f"{img_name}_{name}.jpg"
        output_image_path = os.path.join(final_images_folder, output_image_filename)
        transformed_img.save(output_image_path)

        # Copiar el mismo label de la imagen original
        output_label_filename = f"{img_name}_{name}.txt"
        output_label_path = os.path.join(final_labels_folder, output_label_filename)
        shutil.copy(label_path, output_label_path)


In [34]:
def unified_augmentation(input_folder, labels_folder, final_images_folder, final_labels_folder):
    images = [os.path.join(input_folder, file) for file in os.listdir(input_folder) if file.endswith('.jpg')]

    for img_path in images:
        img = Image.open(img_path)
        img_name = os.path.basename(img_path).split('.')[0]
        label_path = os.path.join(labels_folder, f"{img_name}.txt")

        # Copiar la imagen y el label original a la carpeta final antes de la augmentación
        shutil.copy(img_path, os.path.join(final_images_folder, f"{img_name}.jpg"))
        shutil.copy(label_path, os.path.join(final_labels_folder, f"{img_name}.txt"))

        # Leer la clase de la etiqueta
        labels = read_label_file(label_path)

        # Verificar si 'traffic_light_yellow' está presente en las etiquetas
        is_traffic_light_yellow = 'traffic_light_yellow' in labels

        # Primera fase de augmentación: aplicar rotaciones
        angles = [90, 180, 270]
        for angle in angles:
            rotated_img = img.rotate(angle)
            rotated_img_name = f"{img_name}_rot_{angle}"

            # Guardar imagen rotada
            rotated_img_path = os.path.join(final_images_folder, f"{rotated_img_name}.jpg")
            rotated_img.save(rotated_img_path)

            # Copiar el label original a la imagen rotada
            shutil.copy(label_path, os.path.join(final_labels_folder, f"{rotated_img_name}.txt"))

            # Aplicar augmentaciones de la segunda fase solo si el label es 'traffic_light_yellow'
            if is_traffic_light_yellow:
                apply_augmentations(rotated_img, rotated_img_name, label_path, final_images_folder, final_labels_folder)

        # Aplicar augmentaciones de la segunda fase a la imagen original
        apply_augmentations(img, img_name, label_path, final_images_folder, final_labels_folder)

    # Contar las imágenes resultantes
    final_image_count = len(os.listdir(final_images_folder))
    print(f"Cantidad final de imágenes procesadas: {final_image_count}")

In [35]:
unified_augmentation(input_folder, labels_folder, final_images_folder, final_labels_folder)

Cantidad final de imágenes procesadas: 42351


In [36]:
# Directorio de labels
labels_folder = 'datasets/cropped_and_labeled_final/labels'

# Crear un conjunto para almacenar las etiquetas únicas
unique_labels = set()

# Leer todos los archivos .txt en el directorio de labels
for label_filename in os.listdir(labels_folder):
    label_path = os.path.join(labels_folder, label_filename)

    # Verificar si es un archivo .txt
    if label_filename.endswith('.txt'):
        with open(label_path, 'r') as label_file:
            # Leer el contenido del archivo (cada archivo puede contener una o más etiquetas)
            label_data = label_file.read().strip()
            # Agregar la etiqueta al conjunto (se evita la duplicación automáticamente)
            unique_labels.add(label_data)

# Convertir el conjunto en una lista para obtener un arreglo
unique_labels_list = list(unique_labels)

# Imprimir las etiquetas únicas
print("Etiquetas únicas presentes en los archivos .txt:")
print(unique_labels_list)

Etiquetas únicas presentes en los archivos .txt:
['traffic_light_red', 'traffic_light_green', 'stop sign', 'traffic_light_yellow', 'pedestrian_traffic_light_red', 'stop', 'pedestrian_traffic_light_green']
